In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [28]:
train_dir = 'C:/Users/DELL/Downloads/New_folder/SEC_Five/dataset/train' 
test_dir = 'C:/Users/DELL/Downloads/New_folder/SEC_Five/dataset/test'

In [29]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [30]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [31]:
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='binary',  # Use 'binary' for binary classification
    shuffle=False)

Found 12017 images belonging to 2 classes.


In [32]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='binary',  # Use 'binary' for binary classification
    shuffle=False)

Found 216 images belonging to 2 classes.


In [33]:
# Define the CNN model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3)),  # Additional layer for better feature extraction
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\mf\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\mf\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


376/376 ━━━━━━━━━━━━━━━━━━━━ 22s 59ms/step


In [35]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


In [36]:
# Use Decision Tree classifier instead of SVM
dt_classifier = DecisionTreeClassifier(
    criterion='gini',         
    max_depth=10  # Increased depth for better accuracy
)
dt_classifier.fit(train_features, train_labels)

DecisionTreeClassifier(max_depth=10)

In [37]:
# Make predictions on the test data
test_predictions = dt_classifier.predict(test_features)
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 65.74%


In [38]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor_binary.h5')
# Save the trained Decision Tree model
joblib.dump(dt_classifier, 'dt_classifier_binary.pkl')

['dt_classifier_binary.pkl']

In [40]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved models
cnn_model = load_model('cnn_feature_extractor_binary.h5')
dt_classifier = joblib.load('dt_classifier_binary.pkl')

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for dt

    # Classify the features with the dt model
    prediction = dt_classifier.predict(features)
    
    # Interpret the prediction for binary classes
    class_labels = {0: 'Male', 1: 'Female'}  # Updated for binary classification
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = r'C:/Users/DELL/Downloads/New_folder/SEC_Five/dataset/test/female/113037.jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
The image is classified as: Female
